# Embedding Generator: Building the Vector Representations

## Overview
This notebook generates **two types of embeddings** for the career skills knowledge graph:
1. **Text Embeddings** (384-dim): Capture semantic meaning of skills and job roles
2. **Graph Embeddings** (128-dim): Capture structural patterns and relationships

## What This Notebook Does:

### Phase 1: Text Embeddings (Semantic Layer)
- Generates rich text descriptions for each node (skills, jobs, skillsets)
- Uses SentenceTransformer (`all-MiniLM-L6-v2`) to create 384-dim vectors
- Enables semantic similarity search (e.g., "Python" ≈ "Programming")

### Phase 2: Graph Embeddings (Structural Layer)
- Uses Node2Vec algorithm to learn structural patterns
- Creates 128-dim vectors based on graph topology
- Captures skill co-occurrence and relationship patterns

### Phase 3: Testing & Export
- Tests hybrid retrieval combining both embedding types
- Saves embeddings to `knowledge_graph/` for use in GraphRAG system

## Input:
- `knowledge_graph/career_skills_kg_latest.pkl` (175 nodes, 1558 edges)

## Output:
- `knowledge_graph/text_embeddings.pkl` (175 × 384 matrix)
- `knowledge_graph/graph_embeddings.pkl` (175 × 128 matrix)

## Key Techniques:
- **SentenceTransformers**: Pre-trained transformer model for text embeddings
- **Node2Vec**: Random walk-based graph embedding algorithm
- **Hybrid Weighting**: Combine semantic + structural similarity with alpha parameter

In [1]:
# Import required libraries
import pickle
import networkx as nx
import numpy as np
from typing import List, Dict, Any, Tuple, Optional
from collections import defaultdict
import json
from karateclub import Node2Vec

# For embeddings and LLM
try:
    from openai import OpenAI
    OPENAI_AVAILABLE = True
except ImportError:
    OPENAI_AVAILABLE = False
    print("OpenAI not installed. Install with: pip install openai")

# For embeddings alternative
try:
    from sentence_transformers import SentenceTransformer
    SENTENCE_TRANSFORMERS_AVAILABLE = True
except ImportError:
    SENTENCE_TRANSFORMERS_AVAILABLE = False
    print("SentenceTransformers not installed. Install with: pip install sentence-transformers")

print("Setup complete!")

/opt/anaconda3/envs/ISR/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Setup complete!


## Load Knowledge Graph

Load the career skills knowledge graph to understand:
- **Node types**: JobCategory (5), Skill (62), Skillset (108)
- **Edge types**: REQUIRES, CO_OCCURS_WITH, LEADS_TO, CHARACTERISTIC_SKILLSET, PART_OF
- **Node attributes**: Frequency, support, lift, confidence scores from association rule mining

This graph contains relationships extracted from real job postings using Apriori algorithm.

## Graph Retrieval Functions

These functions enable intelligent retrieval from the knowledge graph.

In [2]:
# Load the knowledge graph
kg_path = 'knowledge_graph/career_skills_kg_latest.pkl'

with open(kg_path, 'rb') as f:
    kg = pickle.load(f)

print(f"✓ Loaded knowledge graph from {kg_path}")
print(f"  - Nodes: {len(kg.nodes())}")
print(f"  - Edges: {len(kg.edges())}")
print()

# Analyze graph structure
node_types = defaultdict(int)
edge_types = defaultdict(int)

for node, data in kg.nodes(data=True):
    node_type = data.get('node_type', 'unknown')
    node_types[node_type] += 1

for u, v, key, data in kg.edges(keys=True, data=True):
    edge_type = data.get('relationship', 'unknown')
    edge_types[edge_type] += 1

print("Node types:")
for ntype, count in node_types.items():
    print(f"  {ntype}: {count}")

print("\nEdge types:")
for etype, count in edge_types.items():
    print(f"  {etype}: {count}")

✓ Loaded knowledge graph from knowledge_graph/career_skills_kg_latest.pkl
  - Nodes: 175
  - Edges: 1558

Node types:
  JobCategory: 5
  Skill: 62
  Skillset: 108

Edge types:
  REQUIRES: 158
  CHARACTERISTIC_SKILLSET: 108
  CO_OCCURS_WITH: 224
  LEADS_TO: 749
  PART_OF: 319


## Phase 1: Text Embeddings (Semantic Layer)

**Goal**: Convert node information into dense vector representations that capture semantic meaning.

**Why do we need this?**
- Enables finding semantically similar skills (e.g., "Python" ≈ "Java" ≈ "Programming")
- Graph structure alone can't capture conceptual similarity
- Allows fuzzy matching for natural language queries

**Process:**
1. Generate rich text descriptions for each node (combining node type, name, and statistics)
2. Use SentenceTransformer to convert text → 384-dimensional vectors
3. Store embeddings in matrix form (175 nodes × 384 dims)

**Model**: `all-MiniLM-L6-v2` - Fast, efficient transformer model optimized for semantic similarity

## Text Embedding Vector Store

Create embeddings for all nodes to enable semantic similarity search.

In [4]:
### Step 1: Generate Rich Text for Nodes

# Simple function to create a text description for a single node
def generate_node_text(node_id, node_data, graph):
    """
    Create a rich text description for a node.
    This text will capture what the node is and its context.
    """
    node_type = node_data.get('node_type', 'unknown')
    
    # Fix: Use capital letters to match the actual node_type values
    if node_type == 'Skill':  # Changed from 'skill' to 'Skill'
        # For a skill, describe what it is and add statistics
        text = f"Skill: {node_id}"
        
        # Add frequency info if available
        if 'total_frequency' in node_data:
            text += f". Appears in {node_data['total_frequency']} job postings"
        
        if 'global_support' in node_data:
            percentage = node_data['global_support'] * 100
            text += f". Required by {percentage:.1f}% of positions"
        
        return text
    
    elif node_type == 'JobCategory':  # Changed from 'job_category' to 'JobCategory'
        # For a job, describe the role
        text = f"Job Role: {node_id}"
        
        if 'job_count' in node_data:
            text += f". {node_data['job_count']} positions analyzed"
        
        if 'avg_skills_required' in node_data:
            text += f". Typically requires {node_data['avg_skills_required']:.1f} skills"
        
        return text
    
    else:
        # Fallback for other node types
        return f"{node_type}: {node_id}"

# Test it on our Python skill
sample_skill = 'Python'
if sample_skill in kg.nodes():
    node_data = kg.nodes[sample_skill]
    description = generate_node_text(sample_skill, node_data, kg)
    print(f"Generated text:\n{description}")

Generated text:
Skill: Python. Appears in 18 job postings. Required by 36.7% of positions


In [5]:
# Test on different node types to see the descriptions
test_nodes = ['Python', 'Backend Engineer', 'Frontend Engineer']

print("Testing text generation for different nodes:\n")
for node_id in test_nodes:
    if node_id in kg.nodes():
        node_data = kg.nodes[node_id]
        description = generate_node_text(node_id, node_data, kg)
        print(f"✓ {description}\n")

Testing text generation for different nodes:

✓ Skill: Python. Appears in 18 job postings. Required by 36.7% of positions

✓ Job Role: Backend Engineer. 8 positions analyzed. Typically requires 9.1 skills

✓ Job Role: Frontend Engineer. 9 positions analyzed. Typically requires 10.9 skills



## Phase 1.1: Text Embeddings Layer

**Goal**: Create rich text descriptions for each node, then convert them to embeddings (vectors).

**Why?** 
- Text embeddings capture semantic meaning
- Allows us to find "similar" skills/jobs based on meaning, not just graph connections
- Example: "Python" and "Programming" are semantically similar even if not directly connected

**Steps:**
1. Generate descriptive text for each node
2. Convert text to vectors using sentence-transformers
3. Store embeddings for fast lookup

In [6]:
print("hello")

hello


In [7]:
# Initialize the embedding model
if SENTENCE_TRANSFORMERS_AVAILABLE:
    print("Loading SentenceTransformer model...")
    embedding_model = SentenceTransformer('all-MiniLM-L6-v2')  # Fast, efficient model
    print("✓ Model loaded successfully!")
    print(f"  Embedding dimension: {embedding_model.get_sentence_embedding_dimension()}")
else:
    print("⚠ SentenceTransformers not available. Install with: pip install sentence-transformers")
    embedding_model = None

Loading SentenceTransformer model...
✓ Model loaded successfully!
  Embedding dimension: 384


In [8]:
# Generate embeddings for all nodes
node_texts = []
node_ids = []
node_metadata = []

print("Generating text descriptions for all nodes...")
for node_id, node_data in kg.nodes(data=True):
    text = generate_node_text(node_id, node_data, kg)
    node_texts.append(text)
    node_ids.append(node_id)
    node_metadata.append({
        'id': node_id,
        'type': node_data.get('node_type', 'unknown'),
        'data': node_data
    })

print(f"✓ Generated {len(node_texts)} text descriptions")
print(f"\nSample texts:")
for i in range(min(3, len(node_texts))):
    print(f"  {i+1}. {node_texts[i]}")

Generating text descriptions for all nodes...
✓ Generated 175 text descriptions

Sample texts:
  1. Job Role: Backend Engineer. 8 positions analyzed. Typically requires 9.1 skills
  2. Job Role: Frontend Engineer. 9 positions analyzed. Typically requires 10.9 skills
  3. Job Role: Software Engineer. 18 positions analyzed. Typically requires 4.7 skills


In [9]:
# Create embeddings for all node texts
if embedding_model is not None:
    print("Creating embeddings... (this may take a moment)")
    embeddings = embedding_model.encode(node_texts, show_progress_bar=True)
    embeddings = np.array(embeddings)
    
    print(f"✓ Created embeddings!")
    print(f"  Shape: {embeddings.shape}")
    print(f"  ({len(node_ids)} nodes × {embeddings.shape[1]} dimensions)")
else:
    print("Cannot create embeddings without model")
    embeddings = None

Creating embeddings... (this may take a moment)


Batches: 100%|██████████| 6/6 [00:00<00:00, 12.86it/s]

✓ Created embeddings!
  Shape: (175, 384)
  (175 nodes × 384 dimensions)


In [10]:
# Save embeddings and metadata for future use
text_embeddings_data = {
    'embeddings': embeddings,
    'node_ids': node_ids,
    'node_metadata': node_metadata,
    'model_name': 'all-MiniLM-L6-v2',
    'embedding_dim': 384
}

# Save to file
import pickle
save_path = 'knowledge_graph/text_embeddings.pkl'
with open(save_path, 'wb') as f:
    pickle.dump(text_embeddings_data, f)

print(f"✓ Saved text embeddings to {save_path}")
print(f"  Contains: {len(node_ids)} node embeddings")

✓ Saved text embeddings to knowledge_graph/text_embeddings.pkl
  Contains: 175 node embeddings


In [11]:
# Convert text embeddings to dict if needed
if isinstance(embeddings, np.ndarray):
    text_embeddings_dict = {
        node_ids[i]: embeddings[i] 
        for i in range(len(node_ids))
    }
    print("✓ Converted text embeddings to dict")
else:
    text_embeddings_dict = embeddings
    print("✓ Text embeddings already a dict")

# Now both are dicts!
print(f"\nText embeddings dict: {len(text_embeddings_dict)} nodes")
print(f"Sample: text_embeddings_dict['Python'].shape = {text_embeddings_dict['Python'].shape}")

✓ Converted text embeddings to dict

Text embeddings dict: 175 nodes
Sample: text_embeddings_dict['Python'].shape = (384,)


## Phase 1.1 complete : Generate text embeddings and save them

## Phase 2: Graph Embeddings (Structural Layer)

**Goal**: Learn embeddings that capture the graph structure and relationship patterns.

**Why do we need this?**
- Text embeddings miss structural patterns (e.g., which skills frequently co-occur)
- Graph structure reveals job market realities (e.g., Python + SQL commonly required together)
- Captures skill prerequisites and progression paths

**Algorithm: Node2Vec**
- Performs random walks through the graph to explore neighborhoods
- Learns to predict node contexts (similar to Word2Vec for graphs)
- Parameters:
  - `walk_length=80`: How far each walk explores
  - `num_walks=10`: Number of walks per node
  - `dimensions=128`: Output vector size

**Process:**
1. Convert MultiDiGraph → Simple undirected graph (aggregate edges)
2. Map node names → integer IDs (Node2Vec requires numeric nodes)
3. Train Node2Vec model on graph structure
4. Map embeddings back to original node names
5. Output: 175 nodes × 128 dimensions

In [ ]:
from karateclub import Node2Vec

In [12]:
def convert_to_simple_graph(multi_graph):
    """
    Convert MultiDiGraph to a simple undirected Graph.
    Aggregates multiple edges and removes direction.
    """
    simple_graph = nx.Graph()
    
    # Add all nodes
    simple_graph.add_nodes_from(multi_graph.nodes())
    
    # Add edges (aggregate weights if multiple edges exist)
    edge_weights = {}
    for u, v, key, data in multi_graph.edges(keys=True, data=True):
        # Make edge undirected by sorting nodes
        edge = tuple(sorted([u, v]))
        
        # Default weight is 1
        weight = data.get('weight', 1.0)
        
        # Aggregate if edge already exists
        if edge in edge_weights:
            edge_weights[edge] += weight
        else:
            edge_weights[edge] = weight
    
    # Add weighted edges to simple graph
    for (u, v), weight in edge_weights.items():
        simple_graph.add_edge(u, v, weight=weight)
    
    print(f"✓ Converted to simple graph:")
    print(f"  Nodes: {len(simple_graph.nodes)}")
    print(f"  Edges: {len(simple_graph.edges)}")
    
    return simple_graph

# Convert the graph
simple_kg = convert_to_simple_graph(kg)

✓ Converted to simple graph:
  Nodes: 175
  Edges: 938


In [13]:
# Create mapping from node names to numbers (0, 1, 2, ...)
node_to_index = {node: idx for idx, node in enumerate(simple_kg.nodes())}
index_to_node = {idx: node for node, idx in node_to_index.items()}

In [14]:
# Create a new graph with integer node IDs
numbered_graph = nx.Graph()

# Add nodes with integer IDs
numbered_graph.add_nodes_from(range(len(simple_kg.nodes())))

# Add edges using integer IDs
for u, v, data in simple_kg.edges(data=True):
    u_idx = node_to_index[u]
    v_idx = node_to_index[v]
    numbered_graph.add_edge(u_idx, v_idx, **data)

print(f"✓ Created numbered graph:")
print(f"  Nodes: {len(numbered_graph.nodes)} (indexed 0 to {len(numbered_graph.nodes)-1})")
print(f"  Edges: {len(numbered_graph.edges)}")

✓ Created numbered graph:
  Nodes: 175 (indexed 0 to 174)
  Edges: 938


## Phase 3: Test Hybrid Retrieval

Test the **HybridGraphRetriever** with alpha=0.8 (80% graph, 20% text):
- Combines text embeddings (semantic) + graph embeddings (structural)
- Query "Python" should return similar skills, relationships, and neighbors

In [16]:
# Get embeddings from the model
graph_embeddings_matrix = node2vec_model.get_embedding()

print(f"✓ Generated graph embeddings!")
print(f"  Shape: {graph_embeddings_matrix.shape}")
print(f"  ({graph_embeddings_matrix.shape[0]} nodes × {graph_embeddings_matrix.shape[1]} dimensions)")

# Map back to original node names (not numbers!)
graph_embeddings = {
    index_to_node[i]: graph_embeddings_matrix[i]
    for i in range(len(graph_embeddings_matrix))
}

print(f"\n✓ Created graph embeddings dictionary with original node names")
print(f"  Example keys: {list(graph_embeddings.keys())[:3]}")
print(f"  Python's graph embedding shape: {graph_embeddings['Python'].shape}")

✓ Generated graph embeddings!
  Shape: (175, 128)
  (175 nodes × 128 dimensions)

✓ Created graph embeddings dictionary with original node names
  Example keys: ['Backend Engineer', 'Frontend Engineer', 'Software Engineer']
  Python's graph embedding shape: (128,)


### Testing retrieval


## Save Graph Embeddings

Export the graph embeddings (175 × 128) to pickle file for use in GraphRAG system.

In [17]:
# Import and reload the retriever
import importlib
import graphRetriever
importlib.reload(graphRetriever)
from graphRetriever import HybridGraphRetriever

# Create retriever with your data
retriever = HybridGraphRetriever(
    knowledge_graph=kg,
    node_ids=node_ids,
    text_embeddings=embeddings,
    graph_embeddings=graph_embeddings,
    alpha=0.8  # Balanced
)

print("\n" + "="*60)
print("Testing Retriever")
print("="*60)

# Test 1: Query for a skill
docs = retriever.get_relevant_documents("Python")

print(f"\n📄 Retrieved {len(docs)} documents\n")

# Show first few documents
for i, doc in enumerate(docs, 1):
    print(f"{i}. Source: {doc.metadata['source']}")
    print(f"   Node: {doc.metadata.get('node_id', 'N/A')}")
    if 'similarity' in doc.metadata:
        print(f"   Similarity: {doc.metadata['similarity']:.3f}")
    print(f"   Content: {doc.page_content[:80]}...")
    print()

✓ LangChain imports successful!
✓ sklearn imports successful!
✓ LangChain imports successful!
✓ sklearn imports successful!
  ✓ Created 175 weighted embeddings
✓ Retriever created with 175 nodes
  Alpha (α) = 0.8
  Text weight = 20%, Graph weight = 80%

Testing Retriever

🔍 Query: 'Python'
   ✓ Found 1 matching node(s): ['Python']
   ✓ Found 5 similar nodes
   ✓ Added 38 relationships
   ✓ Total documents: 8

📄 Retrieved 8 documents

1. Source: query_match
   Node: Python
   Content: Skill: Python. Appears in 18 job postings. Required by 36.7% of positions...

2. Source: similarity_search
   Node: Angular
   Similarity: 0.813
   Content: Skill: Angular. Appears in 9 job postings. Required by 18.4% of positions...

3. Source: similarity_search
   Node: TensorFlow
   Similarity: 0.797
   Content: Skill: TensorFlow. Appears in 3 job postings. Required by 6.1% of positions...

4. Source: similarity_search
   Node: Terraform
   Similarity: 0.776
   Content: Skill: Terraform. Appears in 3 jo

/var/folders/ch/llzrb2pd6hgf5s9k02bw3cq40000gn/T/ipykernel_8564/2529611516.py:21: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  docs = retriever.get_relevant_documents("Python")


In [18]:
import pickle

save_path = 'knowledge_graph/graph_embeddings.pkl'
with open(save_path, 'wb') as f:
    pickle.dump(graph_embeddings, f)

print(f"✓ Saved graph embeddings to {save_path}")
print(f"  Contains {len(graph_embeddings)} nodes")

✓ Saved graph embeddings to knowledge_graph/graph_embeddings.pkl
  Contains 175 nodes
